In [3]:
import sqlite3

# Create a database connection
conn = sqlite3.connect('library.db')
cursor = conn.cursor()

# Create tables
cursor.execute('''CREATE TABLE IF NOT EXISTS Books (
                    BookID INTEGER PRIMARY KEY,
                    Title TEXT,
                    Author TEXT,
                    ISBN TEXT,
                    Status TEXT)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Users (
                    UserID INTEGER PRIMARY KEY,
                    Name TEXT,
                    Email TEXT)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Reservations (
                    ReservationID INTEGER PRIMARY KEY,
                    BookID INTEGER,
                    UserID INTEGER,
                    ReservationDate TEXT,
                    FOREIGN KEY (BookID) REFERENCES Books(BookID),
                    FOREIGN KEY (UserID) REFERENCES Users(UserID))''')

# Function to add a new book to the database
def add_book():
    title = input("Enter the title of the book: ")
    author = input("Enter the author of the book: ")
    isbn = input("Enter the ISBN of the book: ")
    status = input("Enter the status of the book: ")
    cursor.execute("INSERT INTO Books (Title, Author, ISBN, Status) VALUES (?, ?, ?, ?)",
                   (title, author, isbn, status))
    conn.commit()
    print("Book added successfully.")

# Function to find a book's details based on BookID
def find_book_details():
    book_id = input("Enter the BookID: ")
    cursor.execute("SELECT * FROM Books WHERE BookID=?", (book_id,))
    book = cursor.fetchone()
    if book:
        print("Book ID:", book[0])
        print("Title:", book[1])
        print("Author:", book[2])
        print("ISBN:", book[3])
        print("Status:", book[4])
        cursor.execute("SELECT * FROM Reservations WHERE BookID=?", (book_id,))
        reservation = cursor.fetchone()
        if reservation:
            user_id = reservation[2]
            cursor.execute("SELECT * FROM Users WHERE UserID=?", (user_id,))
            user = cursor.fetchone()
            print("Reserved by:")
            print("User ID:", user[0])
            print("Name:", user[1])
            print("Email:", user[2])
        else:
            print("Not reserved by anyone.")
    else:
        print("Book not found.")

# Function to find a book's reservation status based on BookID, Title, UserID, and ReservationID
def find_reservation_status():
    input_text = input("Enter BookID, Title, UserID, or ReservationID: ")
    if input_text.startswith("LB"):
        # Find by BookID
        book_id = input_text
        cursor.execute("SELECT * FROM Books WHERE BookID=?", (book_id,))
        book = cursor.fetchone()
        if book:
            print("Reservation status:", book[4])
        else:
            print("Book not found.")
    elif input_text.startswith("LU"):
        # Find by UserID
        user_id = input_text
        cursor.execute("SELECT * FROM Users WHERE UserID=?", (user_id,))
        user = cursor.fetchone()
        if user:
            cursor.execute("SELECT * FROM Reservations WHERE UserID=?", (user_id,))
            reservation = cursor.fetchone()
            if reservation:
                book_id = reservation[1]
                cursor.execute("SELECT * FROM Books WHERE BookID=?", (book_id,))
                book = cursor.fetchone()
                print("Reservation status:", book[4])
            else:
                print("No reservation found.")
        else:
            print("User not found.")
    elif input_text.startswith("LR"):
        # Find by ReservationID
        reservation_id = input_text
        cursor.execute("SELECT * FROM Reservations WHERE ReservationID=?", (reservation_id,))
        reservation = cursor.fetchone()
        if reservation:
            book_id = reservation[1]
            cursor.execute("SELECT * FROM Books WHERE BookID=?", (book_id,))
            book = cursor.fetchone()
            print("Reservation status:", book[4])
        else:
            print("Reservation not found.")
    else:
        # Find by Title
        title = input_text
        cursor.execute("SELECT * FROM Books WHERE Title=?", (title,))
        book = cursor.fetchone()
        if book:
            print("Reservation status:", book[4])
        else:
            print("Book not found.")

# Function to find all the books in the database
def find_all_books():
    cursor.execute("SELECT * FROM Books")
    books = cursor.fetchall()
    if books:
        for book in books:
            print("Book ID:", book[0])
            print("Title:", book[1])
            print("Author:", book[2])
            print("ISBN:", book[3])
            print("Status:", book[4])
            print("--------------")
    else:
        print("No books found.")

# Function to modify/update book details based on its BookID
def modify_book_details():
    book_id = input("Enter the BookID of the book you want to modify: ")
    cursor.execute("SELECT * FROM Books WHERE BookID=?", (book_id,))
    book = cursor.fetchone()
    if book:
        new_title = input("Enter the new title (leave blank to keep the same): ")
        new_author = input("Enter the new author (leave blank to keep the same): ")
        new_isbn = input("Enter the new ISBN (leave blank to keep the same): ")
        new_status = input("Enter the new status (leave blank to keep the same): ")
        
        # Update the book details if the input is not blank
        if new_title:
            cursor.execute("UPDATE Books SET Title=? WHERE BookID=?", (new_title, book_id))
        if new_author:
            cursor.execute("UPDATE Books SET Author=? WHERE BookID=?", (new_author, book_id))
        if new_isbn:
            cursor.execute("UPDATE Books SET ISBN=? WHERE BookID=?", (new_isbn, book_id))
        if new_status:
            cursor.execute("UPDATE Books SET Status=? WHERE BookID=?", (new_status, book_id))
        
        conn.commit()
        print("Book details updated successfully.")
    else:
        print("Book not found.")

# Function to delete a book from the database based on its BookID
def delete_book():
    book_id = input("Enter the BookID of the book you want to delete: ")
    cursor.execute("SELECT * FROM Books WHERE BookID=?", (book_id,))
    book = cursor.fetchone()
    if book:
        cursor.execute("DELETE FROM Books WHERE BookID=?", (book_id,))
        conn.commit()
        print("Book deleted successfully.")
    else:
        print("Book not found.")

# Main menu loop
while True:
    print("Library Management System")
    print("1. Add a new book")``
    print("2. Find book details")
    print("3. Find reservation status")
    print("4. Find all books")
    print("5. Modify book details")
    print("6. Delete a book")
    print("0. Exit")
    
    choice = input("Enter your choice: ")
    
    if choice == "1":
        add_book()
    elif choice == "2":
        find_book_details()
    elif choice == "3":
        find_reservation_status()
    elif choice == "4":
        find_all_books()
    elif choice == "5":
        modify_book_details()
    elif choice == "6":
        delete_book()
    elif choice == "0":
        break
    else:
        print("Invalid choice. Please try again.")

# Close the database connection
conn.close()

Library Management System
1. Add a new book
2. Find book details
3. Find reservation status
4. Find all books
5. Modify book details
6. Delete a book
0. Exit


In [2]:
from nltk.corpus import gutenberg
from gensim import corpora, models
import pyLDAvis.gensim

# Load the Alice's Adventures in Wonderland text
alice = gutenberg.sents('carroll-alice.txt')

# Create a dictionary representation of the documents
dictionary = corpora.Dictionary(alice)

# Create a corpus (bag of words) representation of the documents
corpus = [dictionary.doc2bow(text) for text in alice]

# Perform topic modeling
ldamodel = models.LdaModel(corpus, num_topics=3, id2word=dictionary, passes=20)

# Print the topics and their top keywords
topics = ldamodel.print_topics(num_topics=3, num_words=3)
for topic in topics:
    print(topic)

# Visualize the topics using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
vis.css = 'ldavis.v1.0.0.css'  # Update the CSS file reference
vis

(0, '0.067*"," + 0.053*"the" + 0.036*"and"')
(1, '0.107*"\'" + 0.039*"I" + 0.038*","')
(2, '0.051*"\'" + 0.045*"," + 0.040*"the"')


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.153207  0.006954       1        1  53.012718
1     -0.068176 -0.098285       2        1  37.116847
2     -0.085031  0.091332       3        1   9.870436, topic_info=     Term         Freq        Total Category  logprob  loglift
0       '  1577.000000  1577.000000  Default  30.0000  30.0000
14      I   501.000000   501.000000  Default  29.0000  29.0000
273  said   406.000000   406.000000  Default  28.0000  28.0000
12      .   699.000000   699.000000  Default  27.0000  27.0000
19    the  1408.000000  1408.000000  Default  26.0000  26.0000
..    ...          ...          ...      ...      ...      ...
24      a    22.733991   572.617335   Topic3  -4.9979  -0.9107
10     in    14.683642   332.928280   Topic3  -5.4351  -0.8056
132   all    12.516426   160.709529   Topic3  -5.5948  -0.2369
42     of    13.551948   467.110532   Topic3  -5.5153  -1.2244
76    for    11.229877   130.184061   Topic3  -5.7032  -0.1347

[224 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
98        1  0.146802     !
98        2  0.455088     !
98        3  0.403707     !
116       1  0.008062    !'
116       2  0.729582    !'
...     ...       ...   ...
306       2  0.823054   you
306       3  0.140366   you
571       1  0.020322  your
571       2  0.873865  your
571       3  0.101612  your

[326 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])